In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
import sys
import scipy
import math
sys.path.append('/home/eduardo/PycharmProjects/treemap-analysis/code')

import Parser
import TemporalMetrics

In [29]:
def point_hyperbole_dist(x, w, h, a):
    # Distance between a point (w,h) and a hyperbole y = a/4x where a is the area we are trying to reach
    return math.sqrt((x - w/2)**2 + (a/(4*x) - h/2)**2)

In [34]:
# minimize (x-w/2)**2 + (a/(4*x) - h/2)**2, 
w = 0.63122
h = 1.5842
a = 2

result = scipy.optimize.minimize(point_hyperbole_dist, x0=w, args=(w,h,a))
result

      fun: 0.2612562949467907
 hess_inv: array([[ 0.05468903]])
      jac: array([  3.90410423e-06])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([ 0.54106481])

In [35]:
optimum_x = result.x[0]
# Minimum corner travel is 4 times the minimum point-hyperbole distance
min_dist = point_hyperbole_dist(optimum_x, w, h, a) * 4
min_dist

1.0450251797871628